# 퇴근후딴짓 작업형2유형

## (sample) 작업형 2유형

백화점 고객 1년간 구매데이터

성별 예측값(남자일 확률)을 csv파일로 생성
roc-auc평가지표에 따라 채점

In [ ]:
# 라이브러리
import pandas as pd

x_test = pd.read_csv("data/X_test.csv")
x_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# EDA 데이터전처리, 피처링
print(x_train.isnull().sum())
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

x_train = x_train.drop(['cust_id'], axis =1)
cust_id = x_test.pop('cust_id')

# 범주형변수 피처 레이블인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = x_train.select_dtypes('object').columns
for col in cols:
	x_train[col] = le.fit_transform(x_train[col])
	x_test[col] = le.fit_transform(x_test[col])

# 모델링 & 하이퍼파라미터 튜닝
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 2022)
rf.fit(x_train, y_train['gender'])

pred = rf.predict_proba(x_test)

# 결과물 저장
output = pd.DataFrame({'cust_id': cust_id, 'gender': pred[:,1]})
output.to_csv('0000.csv', index = False)

## T2-1. 타이타닉 simple baseline 생존여부 예측모델 만들기
https://www.kaggle.com/code/agileteam/t2-1-titanic-simple-baseline/notebook


학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 


In [ ]:
# 데이터 불러오기 
df = pd.read_csv("../input/titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

In [ ]:
# EDA

X_train.shape, X_test.shape, y_train.shape, y_test.shape
X_train.head()
X_test.head()
y_train.head()
X_train.isnull().sum()
X_train.info()

# 생존비율
y_train.Survived.sum() / y_train.shape[0]

In [ ]:
# 전처리

# 불필요한 컬럼 제거
dropcols = ['PassengerId','Name','Ticket','Cabin']   
x_train = X_train.drop(columns = dropcols)
x_test = X_test.drop(columns = dropcols)
y = y_train["Survived"]

# 데이터타입 변경
x_train.Pclass = x_train.Pclass.astype("object")
x_test.Pclass = x_test.Pclass.astype("object")

# 더미변수화 (원핫인코딩)
x_tr_dum = pd.get_dummies(x_train, drop_first= True)
x_ts_dum = pd.get_dummies(x_test, drop_first = True)

# age변수 결측데이터 제거
x_tr_dum.dropna(axis = 0, inplace= True)
y = y[-x_tr_dum.Age.isnull()]
x_ts_dum.dropna(axis = 0, inplace= True)
x_ts_dum.Age = x_ts_dum.Age.astype("int64")

In [ ]:
# 모델링 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 200, max_depth = 7, random_state = 2021)
rf.fit(x_tr_dum, y)

pred = rf.predict(x_ts_dum)
rf.score(x_tr_dum, y)

# 결과 
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': predictions})
output.head()

## T2-2. Pima Indians Diabetes(피마 인디언 당뇨병)

https://www.kaggle.com/code/agileteam/t2-2-pima-indians-diabetes/notebook

당뇨병 여부 판단
이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 


In [ ]:
# 데이터 불러오기    
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')


In [ ]:
# EDA

X_train.shape, X_test.shape, y_train.shape, y_test.shape
X_train.head()
X_test.head()
y_train.head()
X_train.isnull().sum()
X_test.isna().sum()
X_train.info()
X_train.describe()


In [ ]:
# 데이터 전처리

# 결측값: (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)

print('--------- train data ----------')
print('BloodPressure: ', (X_train.BloodPressure == 0).sum())
print('Glucose: ', (X_train.Glucose == 0).sum())
print('SkinThickness: ', (X_train.SkinThickness == 0).sum())
print('Insulin: ', (X_train.Insulin == 0).sum())
print('BMI: ', (X_train.BMI == 0).sum())

print('--------- test data ----------')
print('BloodPressure: ', (X_test.BloodPressure == 0).sum())
print('Glucose: ', (X_test.Glucose == 0).sum())
print('SkinThickness: ', (X_test.SkinThickness == 0).sum())
print('Insulin: ', (X_test.Insulin == 0).sum())
print('BMI: ', (X_test.BMI == 0).sum())

# train데이터에만 0이 있는 변수의 결측값 삭제, 나머지는 평균으로 대체
# Glucose == 0 삭제
X_train.drop(X_train[X_train.Glucose == 0].index, axis = 0, inplace = True)
y_train.drop(X_train[X_train.Glucose == 0].index, axis = 0, inplace = True)

# 나머지 평균대체-방법1)
X_train.loc[:,cols].replace(0,X_train[cols].mean(), inplace = True)

# 나머지 평균대체-방법2)
X_train[cols] = X_train[cols].replace(0, X_train[cols].mean())

In [ ]:
# id 제외
X_train_id = X_train.pop('id')
X_test_id = X_test.pop('id')


In [ ]:
# Glucose == 0 삭제

idx = X_train[X_train.Glucose == 0].index
X_train.drop(idx, axis = 0, inplace = True)
y_train.drop(idx, axis = 0, inplace = True)

In [ ]:
# 스케일링

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [ ]:
# 모델링

from sklearn.svm import SVC
svc = SVC(random_state=2022)
svc.fit(X_train, y_train.Outcome)
pred = svc.predict(X_test)

In [ ]:
# 오버피팅 되었을 경우에 점수가 잘나올 수 있음 (객관적인 평가 아님, 밸리데이션 데이터로 평가 필요함)
round(svc.score(X_train, y_train['Outcome']) * 100, 2) 

## T2-3. Adult Census Income Tutorial

income: 수익(예측해야하는 값)

In [ ]:
# 타겟 수 확인
y_train['income'].value_counts()

# id 구분
X_train_id = X_train.pop('id')
X_test_id = X_test.pop('id')

# 피처 구분
obj_col = X_train.select_dtypes('object').columns
int_col = X_train.select_dtypes(exclude = 'object').columns

X_train.describe()
X_train[obj_col].describe()   

In [ ]:
# 결측치 처리: 최빈값이 비율이 높으면 최빈값으로, 그렇지 않으면 null값으로 대체

X_train.isnull().sum()
X_test.isnull().sum()
X_train['native.country'].value_counts()

def df_fn(df):
    df.workclass.fillna(df.workclass.mode()[0], inplace = True)
    df['native.country'].fillna(df['native.country'].mode()[0], inplace = True)
    df.occupation.fillna('null', inplace = True)
    return df

X_train = df_fn(X_train)
X_test = df_fn(X_test)

In [ ]:
# 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train[obj_col] = X_train[obj_col].apply(le.fit_transform)
X_test[obj_col] = X_test[obj_col].apply(le.fit_transform)

# 스케일링
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X_train[int_col] = mm.fit_transform(X_train[int_col])
X_test[int_col] = mm.transform(X_test[int_col])

In [ ]:
# 데이터 분리

# target값 변경
y = (y_train['income'] != '<=50K').astype(int)
y[:5]

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.15, random_state=2021)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

In [ ]:
# 모델 및 평가

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state = 2022)
model_tree.fit(X_tr, y_tr)
pred = model_tree.predict(X_val)
print('accuracy score:', (accuracy_score(y_val, pred)))

from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state = 2022)
model_rf.fit(X_tr, y_tr)
pred = model_rf.predict(X_val)
print('accuracy score:', (accuracy_score(y_val, pred)))

In [ ]:
# 예측

pred = model_rf.predict(X_test)

# csv생성
output = pd.DataFrame({'id': X_test_id, 'income':pred})
output.to_csv("000000.csv", index=False)
output.head()

In [ ]:
# 채점

y_test = (y_test['income'] != '<=50K').astype(int)
from sklearn.metrics import accuracy_score
print('accuracy score:', (accuracy_score(y_test, pred)))